# Design principles and ideas

dependencies:
* should rid ourselvs of ASE/JuLIP is not maintained. Alternative would be to pass in necessary arrays through python. Nodlnkd is the main one, which is a list of bonds that can be readily calculated with the neighbourlist function of ASE (perhaps too slow?), Ovito (harder to download, but fast)

* Parallelism: need to return a partial ring_statistics array and then sum

In [1]:
using JuLIP
using ASE

[ Info: Precompiling ASE [51974c44-a7ed-5088-b8be-3e78c8ba416c]
[ Info: Skipping precompilation since __precompile__(false). Importing ASE [51974c44-a7ed-5088-b8be-3e78c8ba416c].


In [2]:
at = Atoms(read_xyz("""varMQ_11Ks_4096_500K_min.xyz"""))
at = Atoms(read_xyz("""aSi_500atom_11Ks_annealed.xyz"""))

Atoms{Float64}(StaticArrays.SVector{3, Float64}[[2.39069138, 2.42041735, 1.85496588], [2.0158482, 0.70970138, 3.45161917], [5.62543507, 1.61573943, 0.78942348], [4.58066378, 3.27941693, 2.05973164], [9.0612224, 1.54711137, 2.50510266], [7.92254053, 1.88390615, 0.49954806], [10.83459653, 0.12846927, 2.86914238], [10.11613662, 3.64782069, 2.97192017], [0.9973343, 4.19617045, 2.53915534], [1.18467445, 6.45147861, 1.78155432]  …  [18.83791407, 16.58807644, 20.65288468], [20.51757624, 15.29600717, 21.6343675], [11.96334995, 21.76499536, 18.56820616], [19.81999296, 18.6393605, 20.19377962], [14.74754159, 18.77335164, 20.28472454], [14.8869553, 20.70020081, 21.60255581], [16.43418313, 19.31400318, 18.71958421], [17.16166486, 20.74203952, 0.03858163], [18.19157463, 20.25789827, 19.86229087], [21.4157931, 18.65542207, 18.48238305]], StaticArrays.SVector{3, Float64}[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0

In [3]:
pos = at.X
c = at.cell
inds = [1:size(at.X)[1];]
""

""

In [4]:
size(pos)

(512,)

In [5]:
nl = neighbourlist(at, 2.85)

PairList{Float64, Int32}(StaticArrays.SVector{3, Float64}[[2.39069138, 2.42041735, 1.85496588], [2.0158482, 0.70970138, 3.45161917], [5.62543507, 1.61573943, 0.78942348], [4.58066378, 3.27941693, 2.05973164], [9.0612224, 1.54711137, 2.50510266], [7.92254053, 1.88390615, 0.49954806], [10.83459653, 0.12846927, 2.86914238], [10.11613662, 3.64782069, 2.97192017], [0.9973343, 4.19617045, 2.53915534], [1.18467445, 6.45147861, 1.78155432]  …  [18.83791407, 16.58807644, 20.65288468], [20.51757624, 15.29600717, 21.6343675], [11.96334995, 21.76499536, 18.56820616], [19.81999296, 18.6393605, 20.19377962], [14.74754159, 18.77335164, 20.28472454], [14.8869553, 20.70020081, 21.60255581], [16.43418313, 19.31400318, 18.71958421], [17.16166486, 20.74203952, 0.03858163], [18.19157463, 20.25789827, 19.86229087], [21.4157931, 18.65542207, 18.48238305]], [21.848854069638442 0.0 0.0; 0.0 21.848854069638442 0.0; 0.0 0.0 21.848854069638442], 2.85, Int32[1, 1, 1, 1, 2, 2, 2, 2, 3, 3  …  510, 510, 511, 511, 511

In [6]:
nodlnkd = [nl.j[nl.i .== m] for m in 1:length(pos)]
""

""

In [7]:
lnks = [length(nodlnkd[m]) for m in 1:length(pos)]
""

""

# Dijkstra

In [8]:
function pass
end

pass (generic function with 0 methods)

In [9]:
function dijkstra_nonwgt(nodsrc, lvlreq, lnks, nodlnkd, verbose=false)
    if !verbose
        println = pass
    end
    # initialise
    lvldist = lvlreq + 100 .+ zeros(Int64, length(lnks))
    lvldist[nodsrc] = 0

    quebgn = 0
    quend = 1
    
    queue = zeros(Int64, length(lnks))
    queue[1] = nodsrc

    while quebgn < quend
        # println("start next node")
        quebgn += 1
        nodcrt = queue[quebgn]
        # println("nodcrt is ", nodcrt)
        lvlprb = lvldist[nodcrt] + 1
        # println("lvlprb is ", lvlprb)
        for lnlscrt = 1:lnks[nodcrt]
            nodprb = nodlnkd[nodcrt][lnlscrt]
            # println("nodprb is ", nodprb)
            if lvldist[nodprb] > lvlprb
                lvldist[nodprb] = lvlprb
                if lvlprb < lvlreq
                    quend += 1
                    queue[quend] = nodprb
                    # println(queue[1:25])
                end
            end
        end
    end


    return lvldist
end

dijkstra_nonwgt (generic function with 2 methods)

In [10]:
lvldist = dijkstra_nonwgt(1, 10, lnks, nodlnkd)

512-element Vector{Int64}:
 0
 1
 2
 1
 4
 3
 5
 5
 1
 2
 3
 3
 2
 ⋮
 7
 7
 6
 6
 6
 5
 8
 7
 7
 6
 6
 4

In [11]:
lvlref = dijkstra_nonwgt(511, 10, lnks, nodlnkd)

512-element Vector{Int64}:
 6
 5
 7
 7
 6
 7
 5
 7
 5
 6
 6
 8
 7
 ⋮
 3
 3
 2
 3
 5
 1
 2
 2
 1
 1
 0
 2

# record shortest-paths

In [12]:
maxlvl = 10
strpth = zeros(Int, Int(1e5), Int(maxlvl/2)) .- 1
strpth[1,:] .= 0

5-element view(::Matrix{Int64}, 1, :) with eltype Int64:
 0
 0
 0
 0
 0

In [13]:
maxlvl = 20
maxnodes = 1e5
mxlinks = 1e6
mxpaths = 1e5
strpthx = zeros(Int, Int(maxlvl/2))
strpth =  zeros(Int, Int(maxlvl/2))
lvldist = dijkstra_nonwgt(1, 10, lnks, nodlnkd)


function strpth_record(nodcrt, lvlcrt, lvlprim, pths, strpth, strpthx)
    # global pths
    # global strpth
    # global strpthx
    # println("Doing strpth_record")
    for lnkscrt in 1:lnks[nodcrt]
        nodprb = nodlnkd[nodcrt][lnkscrt]
        lvlprb = lvldist[nodprb]
        # @show nodcrt, lvlprb, lvlcrt, lnkscrt
        if lvlprb == 0
            pths += 1
            for lvl in 1:lvlprim
                strpth[pths, lvl] = strpthx[lvl]
            end
        elseif lvlprb == lvlcrt - 1
            strpthx[lvlprb] = nodprb
            pths, strpth, strpthx = strpth_record(nodprb, lvlprb, lvlprim, pths, strpth, strpthx)
        end
    end
return pths, strpth, strpthx
end

strpth_record (generic function with 1 method)

In [14]:
pths = 0
strpthx = zeros(Int, Int(maxlvl/2))
strpth = zeros(Int, 10, Int(maxlvl/2))
# nodcrt, lvlcrt, pths, lvlprim, lvldist, strpth
i=240
pths, strpth, strpthx = strpth_record(i, lvldist[i], lvldist[i], pths, strpth, strpthx)

(2, [105 243 … 0 0; 105 106 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], [105, 106, 247, 246, 0, 0, 0, 0, 0, 0])

In [15]:
@show pths
strpth


pths = 2


10×10 Matrix{Int64}:
 105  243  244  246  0  0  0  0  0  0
 105  106  247  246  0  0  0  0  0  0
   0    0    0    0  0  0  0  0  0  0
   0    0    0    0  0  0  0  0  0  0
   0    0    0    0  0  0  0  0  0  0
   0    0    0    0  0  0  0  0  0  0
   0    0    0    0  0  0  0  0  0  0
   0    0    0    0  0  0  0  0  0  0
   0    0    0    0  0  0  0  0  0  0
   0    0    0    0  0  0  0  0  0  0

In [16]:
@show view(strpth, 1:4, 1:5)

view(strpth, 1:4, 1:5) = [105 243 244 246 0; 105 106 247 246 0; 0 0 0 0 0; 0 0 0 0 0]


4×5 view(::Matrix{Int64}, 1:4, 1:5) with eltype Int64:
 105  243  244  246  0
 105  106  247  246  0
   0    0    0    0  0
   0    0    0    0  0

In [382]:
function prime_ring(iodd, lvlprim, lnks, lvldist, ringstat, nodlknkd, lvlref, querng, srtpth1, srtpth2, prime1, prime2, nodsrc)
    global goal_found, limit, lvlref, ngf, rings
    rngs = size(querng)[1]
    # @show rngs
    for irng in 1:rngs
        # @show irng
        pth1 = querng[irng][1]
        pth2 = querng[irng][2]
        cond = (nodsrc==214 && 2*lvlprim+iodd == 9 && prime1 in [101, 212])
        if pth1 > 0
            for lvlmax in lvlprim:lvlprim+iodd
                for lvlchk in 1:lvlmax-1
                    if cond && pth2 == 2
                        println("Checking pth2")
                        @show lvlmax, lvlchk
                    end

                    # @show querng, lvlprim
                    # @show lvlmax, lvlchk
                    nodchk=srtpth1[pth1,lvlchk]
                    nodmid=srtpth2[pth2,lvlmax-lvlchk]
                    if cond
                        @show nodchk, nodmid
                    end

                    if nodmid == 0
                        nodmid = prime2
                    end
                    if nodchk == 0
                        nodchk = prime1
                    end
                    # @show view(srtpth1, 1:4, 1:5)
                    # @show view(srtpth2, 1:4, 1:5)

                    # @show nodchk, nodmid
                    limit[1:5, 1] = [lvlref[i][nodmid]+lvlprim-1 for i in 1:5]
                    limit[1:5, 2] = [lvlref[i][nodmid]-lvlprim+1 for i in 1:5]
                    limit[5, 2] = lvldist[nodmid] + lvlprim - 1
                    limit[5, 2] = lvldist[nodmid] - lvlprim + 1
                    if nodsrc==214 && 2*lvlprim+iodd == 9 && prime1 in [101, 212]
                        s1 = view(srtpth1, 1:3, 1:3)
                        s2 = view(srtpth2, 1:3, 1:3)
                        @show querng, pth1, pth2, nodchk, nodmid, s1, s2
                    end
                    pair_search(nodchk, nodmid, limit, goal_found)
                    if goal_found
                        goal_found = false
                        ngf[iodd+1] += 1
                        if cond
                            println("Eliminated using ", (nodchk, nodmid))
                        end
                        # if nodsrc==214 && 2*lvlprim+iodd == 9
                        #     @show nodchk, nodmid
                        # end
                        for irgx in irng+1:rngs
                            p1x = querng[irgx][1]
                            p2x = querng[irgx][2]
                            # @show p1x, p2x
                            if p1x > 0
                                if srtpth1[p1x, lvlchk]==nodchk && srtpth1[p2x, lvlmax-lvlchk]==nodmid
                                    # println("unsetting")
                                    querng[irgx][1] = 0
                                end
                                if srtpth1[p2x, lvlchk]==nodchk && srtpth1[p1x, lvlmax-lvlchk]==nodmid
                                    # println("unsetting")
                                    querng[irgx][1] = 0
                                end
                                if srtpth1[p2x, lvlchk]==nodchk && srtpth2[p1x, lvlmax-lvlchk]==nodmid
                                    # println("unsetting")
                                    querng[irgx][1] = 0
                                end
                                if srtpth2[p2x, lvlchk]==nodchk && srtpth1[p1x, lvlmax-lvlchk]==nodmid
                                    # println("unsetting")
                                    querng[irgx][1] = 0
                                end
                            end
                        end
                        # println("Going to next ring")
                        @goto next_ring
                    end
                end
            end
            rs = 2*lvlprim+iodd
            ringstat[rs] += 1 #1/(iodd+1)
            lr = true
            if cond
                println("legit ring found of size ", rs)
            end
            # @show vcat([[nodsrc,], pth1, pth2, [prime1,], [prime2,]]...)
            # println("here")
            r = collect(Iterators.flatten(
                [[nodsrc,], srtpth1[pth1, 1:Int((rs-(2+iodd))/2)], [prime1,], [prime2,], srtpth2[pth2, 1:Int((rs-(2+iodd))/2)]]))
            if cond
                println(r)   
            end
            push!(rings[rs],
                r
            )
        end 
    # println("Jumped here")    
    @label next_ring
    end
return 
end



prime_ring (generic function with 2 methods)

In [280]:
strpth[1, :]

5-element Vector{Int64}:
 506
 511
   0
   0
   0

In [335]:
function pair_search(nodcrt, nodgoal, limit, goal_found)
    # Input: goal_found, nodcrt, nodgoal, lnks(), lvldist(), nodlnkd(), lvlref(), limit()
    # Output: goal_found
    # Internal: lmtx()
    global goal_found, lnks, nodlnkd, lvlref, lvldist
    # print("Doing pair_search")
    
    if nodcrt == nodgoal
        # Goal found
        # println("Goal found!")
        goal_found = true
        return
    end
    # @show nodcrt
    for lnkscrt in 1:lnks[nodcrt]
        next_search = false
        
        nodprb = nodlnkd[nodcrt][lnkscrt]
        # @show lnkscrt, nodprb

        for iref in 1:4
            # @show lvlref[iref][nodprb], limit
            if lvlref[iref][nodprb] >= limit[iref, 1] || lvlref[iref][nodprb] <= limit[iref, 2]
                # println("going to next search stage 1 at iref ", iref)
                @goto next_search
            end
        end
        
        if lvldist[nodprb] >= limit[5, 1] || lvldist[nodprb] < limit[5, 2]
            # println("going to next search stage 2")
            @goto next_search
            # continue
        end
        
        lmtx = zeros(Int, 5, 2)
        for iref in 1:5
            lmtx[iref, 1] = limit[iref, 1]
            lmtx[iref, 2] = limit[iref, 2] + 1
        end
        # println("Pair searching one deeper")
        pair_search(nodprb, nodgoal, lmtx, goal_found)
        
        if goal_found
            return
        end
        @label next_search
    end
    
end


pair_search (generic function with 1 method)

In [157]:
limit = zeros(Int, 5, 2)

lvlref = [dijkstra_nonwgt(i, 10, lnks, nodlnkd) for i in [50, 150, 250, 400, 1]]
# need to initialise limit
nodmid = 239
lvlprim = lvldist[nodmid]
@show lvlprim
limit[:, 1] = [lvlref[i][nodmid]+lvlprim-1 for i in 1:5]
limit[:, 2] = [lvlref[i][nodmid]-lvlprim+1 for i in 1:5]
limit[5, 1] = lvldist[nodmid] + lvlprim - 1
limit[5, 2] = lvldist[nodmid] - lvlprim + 1
@show limit
goal_found = false
mxnodes = Int(1e4)
nodcrt = 1
lvlcrt = lvldist[nodcrt]
pair_search(nodcrt, 5, limit, goal_found)

lvlprim = 4
limit = [11 5; 8 2; 6 0; 8 2; 7 1]
Doing pair_searchnodcrt = 1
(lnkscrt, nodprb) = (1, 2)
((lvlref[iref])[nodprb], limit) = (6, [11 5; 8 2; 6 0; 8 2; 7 1])
((lvlref[iref])[nodprb], limit) = (7, [11 5; 8 2; 6 0; 8 2; 7 1])
((lvlref[iref])[nodprb], limit) = (4, [11 5; 8 2; 6 0; 8 2; 7 1])
((lvlref[iref])[nodprb], limit) = (7, [11 5; 8 2; 6 0; 8 2; 7 1])


LoadError: UndefVarError: `iref` not defined

# Putting together into program

In [185]:
using Base
output_file_path = "rings_out.log"


"rings_out.log"

In [184]:
# Define a custom show method with truncation
function Base.show(io::IO, x::Matrix{T}) where T
    show(IOContext(io, :limit => 20), x)
end

In [284]:
a = collect(Iterators.flatten([[1,2], [2,3], [4,]]))[1:4]

4-element Vector{Int64}:
 1
 2
 2
 3

In [256]:
rings[9], a

(Vector{Int64}[], [1, 2, 2, 3, 4])

In [261]:
push!(rings[9], a)

1-element Vector{Vector{Int64}}:
 [1, 2, 2, 3, 4]

In [259]:
rings[9]

6-element Vector{Vector{Int64}}:
 #undef
 #undef
 #undef
 #undef
 #undef
    [1, 2, 2, 3, 4]

In [383]:
# global parameters
maxlvl = L = 10
ringstat = zeros(Float64, 2*maxlvl)
limit = zeros(Int, 5, 2)
goal_found = false
ngf = [0, 0]
mxnodes = Int(1e4)
mxpths = 10000
rings = [Vector{Vector{Int64}}() for i in range(1,20)]


# open(output_file_path, "w") do io
#     redirect_stdout(io) do

for nodsrc in 1:512 #length(pos)
    # flush(stdout)
    # flush(io)
    # @show nodsrc
    # for one node
    lvlref = [dijkstra_nonwgt(i, 30, lnks, nodlnkd) for i in [50, 150, 250, 400, 1]]

    # nodsrc = 1
    lvldist = dijkstra_nonwgt(nodsrc, maxlvl, lnks, nodlnkd)

    # find all the prime-mid-nodes
    primes = findall(x -> x<(L-maxlvl%2)/2 && x>1, lvldist)
    prime_mid_nodes = []

    # check each prime-mid-node
    primes_done = []
    for prime in primes
        # even ring case here
        
        if size(findall([lvldist[n] == lvldist[prime]-1 for n in nodlnkd[prime]]))[1] > 1
            # println("checking even prime ", prime)
            push!(prime_mid_nodes, prime)
            
            # if nodlnkd[prime] in primes
            
            # find shortest paths to source from this prime-mid-node
            pths = 0
            strpth = zeros(Int, mxpths, Int(maxlvl/2))
            strpthx = zeros(Int, Int(maxlvl/2))
            querng = []
            
            # @show prime, lvldist[prime]
            pths, strpth, strpthx = strpth_record(prime, lvldist[prime], lvldist[prime], pths, strpth, strpthx)
            # @show pths, strpth
            
            # check if there is a ring containing this prime-mid-node and source
            for pth1 in 1:pths
                for pth2 in pth1:pths
                    inter = intersect(strpth[pth1], strpth[pth2])
                    # @show inter
                    if intersect(strpth[pth1], strpth[pth2]) in [[0], []]
                        # println("pushing ", [prime, strpth[pth1, :], strpth[pth2, :], nodsrc])
                        push!(querng, [pth1, pth2])
                    end
                end
            end

            # check if is prime ring
            # @show querng
            prime_ring(0, lvldist[prime], lnks, lvldist, ringstat, nodlnkd, lvlref, querng, strpth, strpth, prime, prime, nodsrc)
            # @show querng

        end

        # odd ring case here
        equal_lvls = findall([lvldist[n] == lvldist[prime] for n in nodlnkd[prime]])
        # @show equal_lvls
        if size(equal_lvls)[1] > 0
            # println("checking odd prime ", prime)
            push!(prime_mid_nodes, prime)

            prime1 = prime
            # find shortest paths to source from this prime-mid-node
            pths = 0
            strpth = zeros(Int, mxpths, Int(maxlvl/2))
            strpthx = zeros(Int, Int(maxlvl/2))
            
            # @show prime, lvldist[prime]
            pths, strpth, strpthx = strpth_record(prime1, lvldist[prime1], lvldist[prime1], pths, strpth, strpthx)
            # @show pths, strpth
            for prime2 in nodlnkd[prime1][equal_lvls]
                # @show prime2
                if prime2 < prime1
                    continue
                end
                # if prime2 in primes_done
                #     println("prime2 already done ", prime2)
                #     continue
                # end
                # push!(primes_done, prime2)
                querng = []
                
                pths2 = 0
                strpth2 = zeros(Int, mxpths, Int(maxlvl/2))
                strpthx2 = zeros(Int, Int(maxlvl/2))
                pths2, strpth2, strpthx2 = strpth_record(prime2, lvldist[prime2], lvldist[prime2], pths2, strpth2, strpthx2)

                # check both sets of shortest paths for rings
                # @show pths, pths2
                for pth1 in 1:pths
                    for pth2 in 1:pths2
                        inter = intersect(strpth[pth1], strpth2[pth2])
                        # @show inter
                        if inter in [[0], []]
                            # println("pushing ", [prime, strpth[pth1, :], strpth2[pth2, :], nodsrc])
                            push!(querng, [pth1, pth2])
                        end
                    end
                end 
                
                # check if is prime ring
                # display(strpth2[1:3])
                # @show lvldist[prime1], lvldist[prime2]
                # @show querng
                prime_ring(1, lvldist[prime], lnks, lvldist, ringstat, nodlnkd, lvlref, querng, strpth, strpth2, prime1, prime2, nodsrc)
                # @show querng
            end
            # push!(primes_done, prime1)


        end
    end
end

# end
# end


(nodchk, nodmid) = (215, 189)
(querng, pth1, pth2, nodchk, nodmid, s1, s2) = (Any[[1, 1], [1, 2], [2, 1], [2, 2]], 1, 1, 215, 189, [215 102 100; 215 235 118; 0 0 0], [191 192 189; 223 222 213; 0 0 0])
(nodchk, nodmid) = (102, 192)
(querng, pth1, pth2, nodchk, nodmid, s1, s2) = (Any[[1, 1], [1, 2], [2, 1], [2, 2]], 1, 1, 102, 192, [215 102 100; 215 235 118; 0 0 0], [191 192 189; 223 222 213; 0 0 0])
(nodchk, nodmid) = (100, 191)
(querng, pth1, pth2, nodchk, nodmid, s1, s2) = (Any[[1, 1], [1, 2], [2, 1], [2, 2]], 1, 1, 100, 191, [215 102 100; 215 235 118; 0 0 0], [191 192 189; 223 222 213; 0 0 0])
(nodchk, nodmid) = (215, 0)
(querng, pth1, pth2, nodchk, nodmid, s1, s2) = (Any[[1, 1], [1, 2], [2, 1], [2, 2]], 1, 1, 215, 212, [215 102 100; 215 235 118; 0 0 0], [191 192 189; 223 222 213; 0 0 0])
(nodchk, nodmid) = (102, 189)
(querng, pth1, pth2, nodchk, nodmid, s1, s2) = (Any[[1, 1], [1, 2], [2, 1], [2, 2]], 1, 1, 102, 189, [215 102 100; 215 235 118; 0 0 0], [191 192 189; 223 222 213; 0 0 0

In [375]:
ngf

2-element Vector{Int64}:
 3514
 7068

In [385]:
[i for i in rings[9] if i[1]==214]


2-element Vector{Vector{Int64}}:
 [214, 215, 102, 100, 101, 212, 191, 192, 189]
 [214, 473, 451, 351, 325, 344, 215, 102, 103]

In [377]:
[sort(collect(i)) for i in rings[9] if i[1]==214]

2-element Vector{Vector{Int64}}:
 [100, 101, 102, 189, 191, 192, 212, 214, 215]
 [102, 103, 214, 215, 325, 344, 351, 451, 473]

In [386]:
r_set = [Set(i) for i in rings[9]]
srcs = [i[1] for i in rings[9]]

107-element Vector{Int64}:
  10
  11
  26
  29
  67
  72
  74
  75
  82
  83
  83
  85
  86
   ⋮
 423
 435
 436
 451
 458
 471
 473
 480
 496
 496
 508
 510

In [387]:
# Initialize dictionaries to store duplicates and non-duplicates
duplicate_sets = Dict{Set{Int}, Vector{Int}}()
non_duplicate_sets = Dict{Set{Int}, Vector{Int}}()
dup_cts = Dict{Set{Int}, Int}()
set_srcs = Dict{Set{Int}, Set{Int}}()

# Iterate through the array of sets
for (index, s) in enumerate(r_set)
    if haskey(duplicate_sets, s)
        push!(duplicate_sets[s], index)
        push!(set_srcs[s], srcs[index])
        dup_cts[s] += 1
    elseif haskey(non_duplicate_sets, s)
        # println("Moving dups")
        duplicate_sets[s] = non_duplicate_sets[s]
        dup_cts[s] = 2
        push!(duplicate_sets[s], index)
        push!(set_srcs[s], srcs[index])
        delete!(non_duplicate_sets, s)
    else
        non_duplicate_sets[s] = [index]
        set_srcs[s] = Set([srcs[index]])
    end
end

# Extract sets with duplicates and non-duplicates
sets_with_duplicates = [key for (key, value) in duplicate_sets]
sets_without_duplicates = [key for (key, value) in non_duplicate_sets]

# Print results
println("Sets with duplicates:")
for s in sets_with_duplicates
    println(sort(collect(s)), " ", dup_cts[s], " Srcs \n", sort(collect(set_srcs[s])), "\n\n")
end

println("\nSets without duplicates:")
for s in sets_without_duplicates
    println(sort(collect(s)))
end

Sets with duplicates:
[72, 74, 75, 83, 93, 96, 103, 325, 344] 9 Srcs 
[72, 74, 75, 83, 93, 96, 103, 325, 344]


[82, 83, 102, 121, 215, 325, 351, 352, 376] 7 Srcs 
[82, 83, 121, 215, 351, 352, 376]


[67, 85, 86, 92, 321, 322, 341, 342, 471] 9 Srcs 
[67, 85, 86, 92, 321, 322, 341, 342, 471]


[102, 103, 214, 215, 325, 344, 351, 451, 473] 9 Srcs 
[102, 103, 214, 215, 325, 344, 351, 451, 473]


[10, 11, 29, 116, 117, 124, 125, 146, 292] 9 Srcs 
[10, 11, 29, 116, 117, 124, 125, 146, 292]


[160, 408, 409, 411, 416, 417, 435, 436, 437] 7 Srcs 
[160, 408, 411, 416, 417, 435, 436]


[192, 198, 199, 200, 208, 222, 223, 227, 480] 9 Srcs 
[192, 198, 199, 200, 208, 222, 223, 227, 480]


[252, 253, 275, 401, 402, 403, 423, 508, 510] 9 Srcs 
[252, 253, 275, 401, 402, 403, 423, 508, 510]


[100, 101, 102, 189, 191, 192, 212, 214, 215] 9 Srcs 
[100, 101, 102, 189, 191, 192, 212, 214, 215]


[191, 192, 200, 208, 219, 222, 227, 229, 458] 7 Srcs 
[191, 192, 208, 219, 222, 229, 458]


[101, 118, 189, 19

In [189]:
ngf

2-element Vector{Int64}:
  3514
 14141

In [301]:
sf = [i for i in range(1,20)]
sf[1:2:20] = sf[1:2:20]

10-element Vector{Int64}:
  1
  3
  5
  7
  9
 11
 13
 15
 17
 19

In [182]:
(217-1)/9

24.0

In [302]:
ringstat

20-element Vector{Float64}:
    0.0
    0.0
    0.0
   20.0
  960.0
 2874.0
 1801.0
  480.0
  107.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0

In [303]:
norm_ringstat = zeros(Float64, 20)
norm_ringstat[2:2:20] .= ringstat[2:2:20]
norm_ringstat[1:2:20] .= ringstat[1:2:20]
# norm_ringstat[5:2:20] .=  (norm_ringstat[5:2:20] - 1)/2
norm_ringstat ./ sf, sum(norm_ringstat ./ sf)

([0.0, 0.0, 0.0, 5.0, 192.0, 479.0, 257.2857142857143, 60.0, 11.88888888888889, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1005.1746031746031)

# Testing ground

In [166]:
# using Plots
# using Distributions

In [20]:
x2 = [x^2 for x in 0:10] .* mean(lnks)

11-element Vector{Float64}:
   0.0
   4.009765625
  16.0390625
  36.087890625
  64.15625
 100.244140625
 144.3515625
 196.478515625
 256.625
 324.791015625
 400.9765625

In [182]:
histogram(lvldist)
plot!(x2)
ylims!(0, 500)

LoadError: UndefVarError: `histogram` not defined